# **Time series data extraction for feature set 3**

Feature set 2 is defined as the features that are not popular in literature but clinically available according to WAVE as a proxy.

As with feature set 1, for all patients in the derived set, time series data for each of these features will be extracted from the chart events table.

The data will then be observed, cleaned and pre-processed as required to train an LSTM model.

We will collect data for both feature set 1, 2 and 3 so we can train the models with a larger number of features

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load all patients
patient_file = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/03_annotated_set/annotation_v03.parquet'

patient_df = pd.read_parquet(patient_file)
patient_df.shape[0]


5970

In [ ]:
# Load variables for extraction
chartevents_path = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/mimic-iv-2.2-raw-data/icu/chartevents.csv'

In [ ]:
time_window_hours = timedelta(hours=6)

In [ ]:
# State the features to be extracted
feature_set_3_dynamic = [
    220210, 220277, 228640, 220235, 223830, 220224, 220228, 223835, 224685, 224686, 224695, 224687,
    224696, 223849, 224419, 220045, 223901, 220739, 220052, 223761, 228096, 220050, 220051, 220546,
    220615, 227457, 220645, 227442, 225668, 220621, 225690, 220545, 225667, 224697, 220227, 220235,
    226062, 220274, 224828, 225674, 226063, 224743, 224842, 220245, 224727, 229661
]
len(feature_set_3_dynamic)

46

In [ ]:
# Make sure all itemids are unique
feature_set_3_dynamic = list(set(feature_set_3_dynamic))
len(feature_set_3_dynamic)

45

In [ ]:
# Function copied from feature set 1 extraction
def process_chunk_incl_null(chunk, patient_dict, itemid_set, time_window):
    # Filter the columns
    chunk = chunk[['subject_id', 'charttime', 'itemid', 'valuenum']].copy()

    # Drop rows where only itemid is null and NOT valuenum
    chunk = chunk.dropna(subset=['itemid'])

    # Convert subject_id to nullable integer type if necessary
    chunk.loc[:, 'subject_id'] = chunk['subject_id'].astype('Int64')

    # Filter by subject_id
    chunk = chunk[chunk['subject_id'].isin(patient_dict.keys())]

    # Filter by itemid
    chunk = chunk[chunk['itemid'].isin(itemid_set)]

    # Apply the time window filter using the patient_dict
    filtered_rows = []
    for idx, row in chunk.iterrows():
        extubation_time = patient_dict[row['subject_id']]['extubation_starttime']
        if (row['charttime'] >= extubation_time - time_window) and (row['charttime'] <= extubation_time):
            filtered_rows.append(row)

    filtered_df = pd.DataFrame(filtered_rows)
    if not filtered_df.empty:
        filtered_df['extubation_starttime'] = filtered_df['subject_id'].map(lambda x: patient_dict[x]['extubation_starttime'])
        filtered_df['extubation_failure'] = filtered_df['subject_id'].map(lambda x: patient_dict[x]['extubation_failure'])

    return filtered_df

In [ ]:
def extract_patient_data_2_incl_null(patient_df, chartevents_file_path, itemids, time_window):
    # Initialize a list to hold the results
    results = []

    # Define the chunk size
    chunksize = 100000

    # Convert itemids to a set for faster lookup
    itemid_set = set(itemids)

    # Convert patient_df to a dictionary for fast lookups
    patient_dict = patient_df.set_index('subject_id').to_dict('index')

    # Track the number of processed rows
    processed_rows = 0

    # Read and process the file in chunks
    for chunk in pd.read_csv(chartevents_file_path, parse_dates=['charttime'], chunksize=chunksize):

      # Process each chunk and filter the data
      filtered_df = process_chunk_incl_null(chunk, patient_dict, itemid_set, time_window)

      # Append the filtered data to the results list
      results.append(filtered_df)

      # Update the processed rows count
      processed_rows += len(chunk)

      # Print the progress every 10,000,000 rows
      if processed_rows % 10000000 == 0:
          print(f"Processed {processed_rows} rows")


    # Concatenate all the results
    result_df = pd.concat(results, ignore_index=True)

    # Check if result_df is empty and handle it accordingly
    if result_df.empty:
        print("Warning: No data found for the given criteria.")
        return pd.DataFrame(), pd.DataFrame() # Return empty DataFrames


    # Extract the data and labels
    data = result_df[['subject_id', 'charttime', 'itemid', 'valuenum']]
    labels = result_df[['subject_id','extubation_failure']]

    return data, labels

In [ ]:
# Extract time series data for feature set 1 and 2 and save to drive
full_data_set_3, full_labels_3 = extract_patient_data_2_incl_null(patient_df, chartevents_path, feature_set_3_dynamic, time_window_hours)

full_data_set_3.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/05_time_series_data_extraction/feature_set_3_results/full_data_3.parquet')
full_labels_3.to_parquet('/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/data_analysis/datasets/05_time_series_data_extraction/feature_set_3_results/full_labels_3.parquet')

Processed 10000000 rows
Processed 20000000 rows
Processed 30000000 rows
Processed 40000000 rows
Processed 50000000 rows
Processed 60000000 rows
Processed 70000000 rows
Processed 80000000 rows
Processed 90000000 rows
Processed 100000000 rows
Processed 110000000 rows
Processed 120000000 rows
Processed 130000000 rows
Processed 140000000 rows
Processed 150000000 rows
Processed 160000000 rows
Processed 170000000 rows
Processed 180000000 rows
Processed 190000000 rows
Processed 200000000 rows
Processed 210000000 rows
Processed 220000000 rows
Processed 230000000 rows
Processed 240000000 rows


In [ ]:
# Count the number of unqiue patients in the extracted data
full_data_set_3['subject_id'].nunique()
print('Number of unique patients in feature set 2: ', full_data_set_3['subject_id'].nunique())

Number of unique patients in feature set 2:  4701


In [ ]:
# Count the number of itemids present
full_data_set_3['itemid'].nunique()
print('Number of unique itemids in feature set 3: ', full_data_set_3['itemid'].nunique())


Number of unique itemids in feature set 3:  43


In [ ]:
feature_set_3_dynamic

[228096,
 227457,
 220546,
 220545,
 225668,
 225667,
 225674,
 220045,
 226062,
 226063,
 223761,
 220050,
 220051,
 220052,
 225690,
 223901,
 229661,
 228640,
 224419,
 224685,
 224686,
 224687,
 220210,
 224695,
 224696,
 224697,
 224828,
 220224,
 220739,
 220228,
 220227,
 220615,
 224842,
 220235,
 220621,
 220245,
 223830,
 224727,
 223835,
 220645,
 224743,
 223849,
 227442,
 220274,
 220277]

In [ ]:
list_3 = list(full_data_set_3['itemid'].unique().astype('int64'))
list_3

[223835,
 224685,
 224686,
 224687,
 224695,
 224697,
 228640,
 220045,
 220210,
 220739,
 223901,
 220277,
 228096,
 223849,
 220050,
 220051,
 220052,
 223761,
 220545,
 220645,
 227442,
 220224,
 220235,
 223830,
 224828,
 220227,
 225667,
 225668,
 220228,
 220546,
 220615,
 220621,
 227457,
 220245,
 229661,
 224696,
 225674,
 220274,
 226062,
 226063,
 224842,
 225690,
 224419]

In [ ]:
# Count occurrences in both lists
list1_count = {num: feature_set_3_dynamic.count(num) for num in set(feature_set_3_dynamic)}
list2_count = {num: list_3.count(num) for num in set(list_3)}

In [ ]:
# Find numbers present in the first list but not in the second
difference_manual = []
for num in list1_count:
    if num not in list2_count or list1_count[num] > list2_count[num]:
        difference_manual.extend([num] * (list1_count[num] - list2_count.get(num, 0)))

# Convert to sorted list
difference_manual_sorted = sorted(difference_manual)
print(difference_manual_sorted)

[224727, 224743]


In [ ]:
len(difference_manual_sorted)

2

There are 2 itemids for which no data has been collected.

We can use the d_items file to identify these.

In [ ]:
# Load the d_items data frame
d_items_file = '/content/drive/MyDrive/MSc_Final_Project/02_data_analysis/mimic/mimic-iv-2.2-raw-data/icu/d_items.csv'
d_items_df = pd.read_csv(d_items_file)
d_items_df.head()

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN


In [ ]:
# Identify and print the label for the itemids that were missing
missing_labels = d_items_df[d_items_df['itemid'].isin(difference_manual_sorted)]['label'].unique()

# Print the itemid next to its label
for label in missing_labels:
    print(f"itemid: {label}")

itemid: Resting Energy Expenditure
itemid: Vd/Vt Ratio


The above two features were not present in the additional feature data.